<a href="https://colab.research.google.com/github/janilles/log_processing/blob/master/CRC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CRC server logs porcessing
NOTE: Do not use Runtime --> Run all 

Processed data is stored in CRC_database Google Sheet on the team's Google Drive.

## Notebook set up and authentication of Google API

In [0]:
#@title Set up including client spreadsheet object
# install gspread library 
!pip install -U -q gspread

# to interact with Google Sheets API
import gspread

# Google API authentication 
from google.oauth2.service_account import Credentials

# used for file upload prompts
from google.colab import files

# used for reading CSVs
import pandas as pd

# used in downloads function for campaign names
import re

# from Google genereated json keyfile
# used in credentials
keyfile_dict = {
}

# used in credetials
# drive scope needed for file management
scope = ['https://spreadsheets.google.com/feeds',
         'https://www.googleapis.com/auth/drive']

# if using json instead of dict change to: .from_service_account_file
credentials = Credentials.from_service_account_info(keyfile_dict, scopes=scope)

# create gspread.client.Client object
gc = gspread.authorize(credentials)

# create client spreadsheet object
sh = gc.open("CRC_database")


# Downloads report

## Upload file from your computer

In [0]:
#@title Downloads file
# upload widget
file = files.upload()

# get file name
for name in file.items():
    downloads_file_name = name[0]

# make sure 'downloads' file has been uploaded
if 'loads' in downloads_file_name:
    # read file
    downloads = pd.read_csv(downloads_file_name)
else:
    print('Are you sure this is a downloads file?')

# preview last five rows 
downloads.tail()


## Response type overview
Just to quickly check the data quality

In [0]:
#@title Rows count summary
downloads['response_type'].value_counts()

In [0]:
#@title Downlaods summary
all_downloads = downloads['requests'].sum()
success_downloads = downloads[downloads['response_type'] == 'Success']['requests'].sum()
error_downloads = all_downloads - success_downloads
per_cent_successful = round((success_downloads/all_downloads)*100, 1)

print(f'{all_downloads} total downloads')
print(f'{success_downloads} succesful downloads')
print(f'{error_downloads} errors when downloading')
print(f'{per_cent_successful}% were successful')


## Format and clean downloads file

In [0]:
# filter out error reponse types
downloads = downloads[downloads['response_type'] != 'Error']

# drop response type column
downloads.drop(columns=['response_type'], inplace=True)

# remove file extensions and 'documnets/' from object name
downloads['object'] = downloads['object'].str.replace(r'documents/|\.zip|\.wav|\.jpg|\.mp4','')

# replace + _ %2B with space
downloads['object'] = downloads['object'].str.replace('\+|_|%2B',' ')

# remove leading dates and typos 'i'
downloads['object'] = downloads['object'].str.lstrip('0123456789. i')

# preview file
downloads.head()


## Add campaigns column

In [0]:
d = {'a':
     'abc|'
     'def'}
d

In [0]:
# create campaign dictionary 
# key = campaign name, value = string or regex to look for
campaigns_dictionary = {
    'Looking Out For Each Other':
        'lookingout|looking out',
    'Stay Home For Your Family':
        'family',
    'Symptoms':
        'symptoms',
    'Trifetica Lockup':
        'trifecta',
    'Age Specific: Anyone Can Get It':
        'age specific|BAME Assets', # BAME needs reviewing
    'Anyone Can Get It':
        '(?=.*anyone can get)^((?!age).)*$', # anyone can get it AND NOT age
    'Door Drop':
        'prime minister|door drop|doordrop',
    'Hand Hygiene':
        'HH|hand hygiene|handhygiene|handwashing',
    'NHS Recources':
        'nhs|children|gp|hospital|pharma|Health At Home Toolkit and Assets',
    'Now More Than Ever EMM':
        'every mind matters|emm',
    'Ports of Entry':
        'ports',
    'Stay At Home Save Lives':
        'save lives|anyone can spread it|' # resources with bad zip file names below
        'DOOH 6 - updated 2020.04.07|Digital Screens - updated 2020.04.07|'
        'Email Signatures - updated 2020.04.07|Radio Resources - updated 2020.04.07|'
        'Social Media Resources - updated 2020.04.07|'
        'Translated Resources - updated 2020.04.16|Web Banners - updated 2020.04.07'
}

# function mapping resource names to campaigns 
def adds_campaign_name(row):
    for key, value in campaigns_dictionary.items():
        if re.search(value, row['object'], re.I) is not None:
            return key

# create a new colum for campaign names
downloads['campaign'] = downloads.apply(lambda row: adds_campaign_name(row), axis=1)

# preview new dataframe
downloads.head()


## Add resource type column

In [0]:
# create resource type dictionary 
# key = resource type, value = string present in log object
resource_type = {
    'Posters': 'poster',
    'Digital Screens': 'digitalscreen|digital screen',
    'Email Signatures': 'emailsig|email sig|esignature',
    'Web Banners': 'webban|web ban|website ban|websiteban|'
        'Age Specific Anyone Can Get It', # bad zip file name factored in
    'Social Media': 'socialmed|social med|social ass|socialass|social post|'
        'BAME Assets', # bad zip file name factored in
    'Radio': 'radio',
    'TV Ad': 'tvad|tv ad',
    'Digital out-of-home': 'digital out of home|digital out-of-home|digital ooh|dooh',
    'Door Drop Resource': 'prime minister|door drop|doordrop',
    'Pull Up Banners': 'pull up banner|pull-up banner',
    'Alternative Formats': 'alternative for|alternativefor',
    'Social Distancing': 'socialdist|social dist',
    'Toolkit': 'toolkit'
}

# function mapping resource types to resource objects 
def adds_resource_type(row):
    for key, value in resource_type.items():
        if re.search(value, row['object'], re.I) is not None:
            return key

# create a new colum for resource type
downloads['resource type'] = downloads.apply(
    lambda row: adds_resource_type(row), axis=1)

# remove empty objects
downloads = downloads[downloads['object'] != '']

# preview new dataframe
downloads.head()


## Send downloads data to Google Sheets
Change worksheet name in the widget below if necessary

In [0]:
#@title Sending to this worksheet tab
downloads_worksheet = 'Downloads_API' #@param {type:"string"}
# create list of lists
lol_downloads = downloads.values.tolist()

# load values to relevant worksheet on the spreadsheet
sh.values_append(
    downloads_worksheet, # worksheet name as it appears on the existing file
    {'valueInputOption': 'USER_ENTERED'}, # as if entered by a person in Google Sheets UI
    {'values': lol_downloads} 
    )


# Pages report

## Upload file from your computer

In [0]:
#@title Page views file
# upload widget
file = files.upload()

# get file name
for name in file.items():
    pages_file_name = name[0]

# make sure 'page' file has been uploaded
if 'age' in pages_file_name:
    # read file
    pages = pd.read_csv(pages_file_name)
else:
    print('Are you sure this is a page views file?')

# preview last five rows 
pages.tail()


## Format and clean page views file

In [0]:
# filter out error reponse types
pages = pages[pages['response_type'] != 'Error']

# drop response type column
pages.drop(columns=['response_type'], inplace=True)

# remove trailing backslash character
pages['path'] = pages['path'].str.rstrip('/')

# drop rows containing these words or characters
pages = pages[~pages['path'].str.contains('favicon|static|prod-assets|\?')]

# merge duplicate path names
pages = pages.groupby(['date', 'path'], as_index=False)['requests'].sum()

# preview file
pages.tail()


## Add campaigns column

In [0]:
# create a list of campaigns
campaigns = ['looking-out-for-each-other',
             'stay-home-for-your-family',
             'symptoms',
             'trifetica-lockup',
             'age-specific',
             'anyone-can-get-it',
             'door-drop-campaign',
             'hand-hygiene',
             'nhs-resources',
             'now-more-than-ever-every-mind-matters',
             'ports-of-entry',
             'stay-home-save-lives',
             'subscribe-updates',
             'phe']

# function mapping URL to campaigns 
def adds_campaign_from_url(row):
    for campaign in campaigns:
        if re.search(campaign, row['path'], re.I) is not None:
            return campaign

# create campaign column
pages['campaign'] = pages.apply(lambda row: adds_campaign_from_url(row), axis=1)

# format campaign column
pages['campaign'] = pages['campaign'].str.replace('-', ' ').str.title()

# capitalise NHS and PHE in campaign column
pages['campaign'] = pages['campaign'].str.replace('Nhs', 'NHS').str.replace('Phe', 'PHE')

# dataframe preview
pages.head()


## Send pages data to Google Shees
Change worksheet name in the widget below if necessary

In [0]:
#@title Sending to this worksheet tab:
pages_worksheet = 'Page_views_API' #@param {type:"string"}
# create list of lists without 'response type' column
lol_pages = pages.values.tolist()

# load values to relevant worksheet on the spreadsheet
sh.values_append(
    pages_worksheet, # worksheet name as it appears on the existing file
    {'valueInputOption': 'USER_ENTERED'}, # as if entered by a person in Google Sheets UI
    {'values': lol_pages} 
    )


# Link web scraping

## Get a set of all pages from Google Sheet database

In [0]:
# create worksheet object for page views tab
wks_pages = gc.open('CRC_database').worksheet('Page_views_API')

# get only values from the second column
values_list = wks_pages.col_values(2)

# create a set of pages 
set_of_pages = set(values_list)

# remove column header from set and unwanted pages
set_of_pages.difference_update({'Page', '/index.html', '/subscribe-updates'})


## BeautifulSoup

In [0]:
import requests
from bs4 import BeautifulSoup

In [0]:
# initiate a dictionary
master = {}

# iterate over all pages and append results to dictionary
for page in set_of_pages:
    try:
        url = 'https://coronavirusresources.phe.gov.uk' + page
        r = requests.get(url)
        html_content = r.text
    except Exception as e:
        pass
    soup = BeautifulSoup(html_content, 'lxml')
    for link in soup.findAll('a', class_='download-button'):
        string = str(link['href'])
        cleaned = string.lstrip('https://downloads.coronavirusresources.phe.gov.uk/documents/')
        master[str(page)] = cleaned


In [0]:
# fromat pandas tables to show full text
pd.set_option('display.max_colwidth', -1)

## Result dictionary to dataframe

In [0]:
data = pd.DataFrame([master])
table = data.T.reset_index()
table.columns = ['page', 'link']
table.head()

## Format and claen - same as download file

In [0]:
# remove file extensions and 'documnets/' from object name
table['link'] = table['link'].str.replace(r'documents/|\.zip|\.wav|\.jpg|\.mp4','')

# replace + _ %2B with space
table['link'] = table['link'].str.replace('\+|_|%2B',' ')

# remove leading dates and typos 'i'
table['link'] = table['link'].str.lstrip('0123456789. i')

# preview file
table.head()


## Create set of resources without campaign and/or type

In [0]:
# create worksheet object for dowbloads tab
wks_download = gc.open('CRC_database').worksheet('Downloads_API')

# create a dataframe
data = wks_download.get_all_values()
headers = data.pop(0)
res_df = pd.DataFrame(data, columns=headers)


In [0]:
# create sliced dataframes
res_no_type = res_df[res_df['Resource type'] == '']
res_no_camp = res_df[res_df['Campaign'] == '']

# create sets to remove duplicates
set_with_no_type = set(res_no_type['Resource'])
set_with_no_camp = set(res_no_camp['Resource'])

In [0]:
set_with_no_type

In [0]:
set_with_no_camp

## Resulting page-resource pairs in a dictionary

In [0]:
results = {}

# choose if you want resource set with no type pairing
# or set with no campaign pairing with pages
for resource in set_with_no_type:
    try:
        pair = table[table['link'] == resource]
        pair.reset_index(inplace=True)
        results[pair['page'][0]] = pair['link'][0]
    except IndexError:
        pass

results

## Send zip file names data to Google Sheets

In [0]:
#@title Sending to this worksheet tab:
bad_names_tab = 'Bad zip file names' #@param {type:"string"}

# create a list of list for sh object below
lol_results = [[k, v] for k,v in results.items()]

# load values to relevant worksheet on the spreadsheet
sh.values_append(
    bad_names_tab, # worksheet name as it appears on the existing file
    {'valueInputOption': 'USER_ENTERED'}, # as if entered by a person in Google Sheets UI
    {'values': lol_results} 
    )
